In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install glob2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from glob import glob
import os
import pandas as pd
path = "/content/drive/MyDrive/Iqram Sir/Fitabase data pulls - seperate"

#Function

In [5]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm.notebook import tqdm

def get_content(patient_folder_adrs):

  post_adrs = sorted(glob(patient_folder_adrs+"/*"))[0]
  

  pre_adrs = sorted(glob(patient_folder_adrs+"/*"))[1]

  for i in glob(post_adrs+"/*"):
    content = i.split("_")
    if "1min" in content:
      post_1min = i
    if "minuteCaloriesNarrow" in content:
      post_cal = i
    if "minuteMETsNarrow" in content:
      post_met = i


  for i in glob(pre_adrs+"/*"):
    content = i.split("_")
    if "1min" in i.split("_"):
      pre_1min = i
    if "minuteCaloriesNarrow" in content:
      pre_cal = i
    if "minuteMETsNarrow" in content:
      pre_met = i
 
  return post_1min, post_cal,post_met, pre_1min, pre_cal, pre_met



def signle_sample(df_1min, df_cal,df_met):
  list_1 = list(df_1min["Time"])
  list_2 = list(df_cal["ActivityMinute"])
  #df_met = mets_minute_refine(df_met,min)
  list_3 = list(df_met["ActivityMinute"])


  hr = []
  aee = []
  time = []
  mets = []

  if len(list_1)>len(list_2):
    for i in tqdm(list_1):
      if i in list_2 and i in list_3:
        hr.append(list(df_1min.query("Time == '"+i+"'")['Value'])[0])
        aee.append(list(df_cal.query("ActivityMinute == '"+i+"'")['Calories'])[0]*4.184)
        mets.append(list(df_met.query("ActivityMinute == '"+i+"'")['METs'])[0])
        time.append(i)

  else:
    for i in tqdm(list_2):
      if i in list_1 and i in list_3:
        hr.append(list(df_1min.query("Time == '"+i+"'")['Value'])[0])
        aee.append(list(df_cal.query("ActivityMinute == '"+i+"'")['Calories'])[0]*4.184)
        mets.append(list(df_met.query("ActivityMinute == '"+i+"'")['METs'])[0])
        time.append(i)

  d={
    "Time" : time,
     "HR(BPM)" : hr,
     "aEE(kJ/min)":aee,
     "METs" : mets
  }

  dataFrame = pd.DataFrame.from_dict(d)
  return dataFrame




from datetime import datetime

def formate24(t):
  time = t.split(" ")
  m2 = time[1]+" "+time[2]
  in_time = datetime.strptime(m2, "%I:%M:%S %p")
  out_time = datetime.strftime(in_time, "%H:%M:%S")
  return out_time


def dif_min(start_time, end_time):
  t1 = datetime.strptime(start_time, "%H:%M:%S")
  t2 = datetime.strptime(end_time, "%H:%M:%S")

  delta = t2 - t1

  return delta.total_seconds()/60





#-------------------------------------------------------------------------------

def range_list(k):
  list_of = []
  main_list = []

  for i in range(len(k)-1):
 
    if k[i+1]-k[i] == 1:
      list_of.append(k[i])
    else:
      list_of.append(k[i])
      main_list.append(list_of)
      list_of=[]
    if i == len(k)-2:
      if k[i+1]-k[i] == 1:
        list_of.append(k[i+1])
        main_list.append(list_of)
      else:
        main_list.append([k[i+1]])
  
  return main_list


def range_list_time(k):
  list_of = []
  main_list = []

  for i in tqdm(range(len(k)-1)):
    if dif_min(formate24(k[i]),formate24(k[i+1])) == 0:
      continue
    if dif_min(formate24(k[i]),formate24(k[i+1])) == 1:
      list_of.append(k[i])
    else:
      list_of.append(k[i])
      main_list.append(list_of)
      list_of=[]
    if i == len(k)-2:
      if dif_min(formate24(k[i]),formate24(k[i+1])) == 1:
        list_of.append(k[i+1])
        main_list.append(list_of)
      else:
        main_list.append([k[i+1]])

  return main_list




def engine(d,tr,max_pick=10):
  first_list = []
  first_met = []
  for i in tr:
    first_list.append((i, list(d.query("Time == '"+i+"'")['Value'])[0]))
    first_met.append(list(d.query("Time == '"+i+"'")['Value'])[0])

  sort_list = sorted(first_met,reverse=True)



  pick_index = []
  for i in range(len(first_list)):
    if first_list[i][-1] in sort_list[0:max_pick]:
      pick_index.append(i)



  p = range_list(pick_index)


  pick_point = []

  for i in p:
    if len(i) > 1:
        pick_point.append(max(first_list[i[0]:i[-1]]))




  main_list = []
  for i in pick_point:

    start_index = first_list.index(i)
    temp_list = []
    for j in range(5):
      if len(first_list) == start_index+j+1: break
      
      if first_list[start_index+j][-1]>first_list[start_index+j+1][-1]:
        temp_list.append(first_list[start_index][-1]-first_list[start_index+j+1][-1])
      else:
        break
    if 3<=len(temp_list)<=5:
      main_list.append(temp_list)

  return main_list




---



---



In [6]:
file_save_destination = "/content/drive/MyDrive/Iqram Sir/hr_recovery/HR_Recovery.csv"

In [7]:

if os.path.exists(file_save_destination)==False:
  dictionary = {
      "Subject":[],
      "Condition":[],
      "Segment":[],
      "1_min_HR_Recovery":[],
      "2_min_HR_Recovery":[],
      "3_min_HR_Recovery":[],
      "4_min_HR_Recovery":[],
      "5_min_HR_Recovery":[],
  }

  df = pd.DataFrame.from_dict(dictionary)
  df.to_csv(file_save_destination,index=False)




for cont in tqdm(sorted(glob(path+"/*"))):
  test_dataset = pd.read_csv(file_save_destination)
  subject =  os.path.basename(cont)
  print("Subject processing: ",subject)
  Post_HR,Post_cal,Post_met, Pre_HR,Pre_cal,Pre_met = get_content(cont)
  

  if 'pre' not in list(test_dataset.query("Subject == '"+subject+"'")['Condition']):
    print()
    print()
    df_pre_hr = pd.read_csv(Pre_HR)
    print("Making Pre Data....")
    print("Refining time ranges....")
    tr = range_list_time(list(df_pre_hr['Time']))
    print("Engine is working on it")
    sub = []
    cond = []

    seg = []
    hr_1 = []
    hr_2 = []
    hr_3 = []
    hr_4 = []
    hr_5 = []
    segment_count = 1
    for index in tqdm(range(len(tr))):
      l = engine(df_pre_hr,tr[index])
      for i,elem in enumerate(l):

        if len(elem) == 3:
          hr_1.append(elem[0])
          hr_2.append(elem[1])
          hr_3.append(elem[2])
          hr_4.append(None)
          hr_5.append(None)
          sub.append(subject)
          cond.append("pre")
          seg.append("Segment_"+str(segment_count))
          segment_count +=1

        if len(elem) == 4:
          hr_1.append(elem[0])
          hr_2.append(elem[1])
          hr_3.append(elem[2])
          hr_4.append(elem[3])
          hr_5.append(None)
          sub.append(subject)
          cond.append("pre")
          seg.append("Segment_"+str(segment_count))
          segment_count +=1

        if len(elem) == 5:
          hr_1.append(elem[0])
          hr_2.append(elem[1])
          hr_3.append(elem[2])
          hr_4.append(elem[3])
          hr_5.append(elem[4])
          sub.append(subject)
          cond.append("pre")
          seg.append("Segment_"+str(segment_count))
          segment_count +=1
        
    dictionary = {
        "Subject":sub,
        "Condition":cond,
        "Segment":seg,
        "1_min_HR_Recovery":hr_1,
        "2_min_HR_Recovery":hr_2,
        "3_min_HR_Recovery":hr_3,
        "4_min_HR_Recovery":hr_4,
        "5_min_HR_Recovery":hr_5,
    }
    temp_df = pd.DataFrame(dictionary)
    temp_df.to_csv(file_save_destination, mode='a', index=False, header=False)



  if 'post' not in list(test_dataset.query("Subject == '"+subject+"'")['Condition']):
      print()
      print()
      df_post_hr = pd.read_csv(Post_HR)
      print("Making Post Data....")
      print("Refining time ranges....")
      tr = range_list_time(list(df_post_hr['Time']))
      print("Engine is working on it")
      sub = []
      cond = []

      seg = []
      hr_1 = []
      hr_2 = []
      hr_3 = []
      hr_4 = []
      hr_5 = []
      segment_count = 1
      for index in tqdm(range(len(tr))):
        l = engine(df_post_hr,tr[index])
        for i,elem in enumerate(l):

          if len(elem) == 3:
            hr_1.append(elem[0])
            hr_2.append(elem[1])
            hr_3.append(elem[2])
            hr_4.append(None)
            hr_5.append(None)
            sub.append(subject)
            cond.append("post")
            seg.append("Segment_"+str(segment_count))
            segment_count +=1

          if len(elem) == 4:
            hr_1.append(elem[0])
            hr_2.append(elem[1])
            hr_3.append(elem[2])
            hr_4.append(elem[3])
            hr_5.append(None)
            sub.append(subject)
            cond.append("post")
            seg.append("Segment_"+str(segment_count))
            segment_count +=1

          if len(elem) == 5:
            hr_1.append(elem[0])
            hr_2.append(elem[1])
            hr_3.append(elem[2])
            hr_4.append(elem[3])
            hr_5.append(elem[4])
            sub.append(subject)
            cond.append("post")
            seg.append("Segment_"+str(segment_count))
            segment_count +=1
          
      dictionary = {
          "Subject":sub,
          "Condition":cond,
          "Segment":seg,
          "1_min_HR_Recovery":hr_1,
          "2_min_HR_Recovery":hr_2,
          "3_min_HR_Recovery":hr_3,
          "4_min_HR_Recovery":hr_4,
          "5_min_HR_Recovery":hr_5,
      }
      temp_df = pd.DataFrame(dictionary)
      temp_df.to_csv(file_save_destination, mode='a', index=False, header=False)
  print("Processing ",subject," Complete....")
  print("------------------------------------------------------")
  print("------------------------------------------------------")


  0%|          | 0/36 [00:00<?, ?it/s]

Subject processing:  001-JV


Making Pre Data....
Refining time ranges....


  0%|          | 0/40178 [00:00<?, ?it/s]

Engine is working on it


  0%|          | 0/298 [00:00<?, ?it/s]

...



Making Post Data....
Refining time ranges....


  0%|          | 0/11294 [00:00<?, ?it/s]

Engine is working on it


  0%|          | 0/100 [00:00<?, ?it/s]

Processing  039-NS  Complete....
------------------------------------------------------
------------------------------------------------------


In [8]:
df = pd.DataFrame(dictionary)
df

,Subject,Condition,Segment,1_min_HR_Recovery,2_min_HR_Recovery,3_min_HR_Recovery,4_min_HR_Recovery,5_min_HR_Recovery
0,039-NS,post,Segment_1,4,7,9,10.0,12.0
1,039-NS,post,Segment_2,2,3,7,8.0,NaN
2,039-NS,post,Segment_3,1,4,7,NaN,NaN
3,039-NS,post,Segment_4,4,6,7,8.0,NaN
4,039-NS,post,Segment_5,7,8,11,NaN,NaN
5,039-NS,post,Segment_6,3,7,14,15.0,NaN
6,039-NS,post,Segment_7,2,4,6,NaN,NaN
7,039-NS,post,Segment_8,1,3,5,7.0,NaN
8,039-NS,post,Segment_9,1,2,4,NaN,NaN
9,039-NS,post,Segment_10,3,5,6,8.0,11.0




---



---

